## Create a general purpose job scraper for www.indeed.com

In [10]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time 
import numpy as np
from random import randint

In [11]:
def get_job_search_url(position, postedDays):
    """Generate a url from position and posted days ago"""

    template = 'https://www.indeed.com/jobs?q={}&fromage={}&limit=50&filter=0'
    url=template.format(position, postedDays)
    
    return url

In [12]:
url = get_job_search_url('machine learning', 1)


##  Extract raw html


In [13]:
response = requests.get(url)

In [14]:
response

<Response [200]>

In [15]:
type(response)

requests.models.Response

In [16]:
#response.text

In [17]:
soup = BeautifulSoup(response.text, 'html.parser')

In [18]:
total_jobs = soup.find('div',{'id':'searchCountPages'}).text.strip()
total_jobs

'Page 1 of 2,340 jobs'

In [20]:
cards = soup.find_all('div', 'jobsearch-SerpJobCard')

In [21]:
len(cards)

54

## Prototype the model with a single record

In [192]:
card = cards[13]

In [139]:
aTag = card.h2.a

In [140]:
job_title = aTag.get('title')
job_title

'Machine Learning Engineer (Remote Contract)'

In [141]:
job_id = card.get('data-jk')
job_id

'4c3a3bfe738fd667'

In [142]:
# To check if the card already stored in the records  # 517da30c1f7ab5f0
type(records)
records_ar = np.array(records)
#'517da30c1f7ab5f0' in np.array(records)
'517da30c1f7ab5f0' in np.array(records)[:,0]

NameError: name 'records' is not defined

In [143]:
#time.sleep(60)
#job_url = 'https://www.indeed.com' + aTag.get('href')
job_url = 'https://www.indeed.com/viewjob?jk=' + job_id
job_url

post_response = requests.get(job_url)
post_soup = BeautifulSoup(post_response.text, 'html.parser')
job_description = post_soup.find('div','jobsearch-jobDescriptionText').text.strip()
job_description

try:
    job_detail = post_soup.find('div','jobsearch-JobDescriptionSection-section').text.strip()
except:
    job_detail = ''

'''items = post_soup.find_all('div','jobsearch-JobDescriptionSection-sectionItem')
for i in items:
    job_detail += '{};'.format(i.text.strip())'''
    

print(job_detail)

req_quals=post_soup.find_all('li','jobsearch-ReqAndQualSection-item')
for r in req_quals:
    job_skills +='{};'.format(r.text.strip())
job_skills

SalaryFrom $90,000 a year;Job TypeFull-time;SalaryFrom $90,000 a year;Job TypeFull-time;Number of hires for this role1;Job TypeContract;


"Bachelor's (Preferred);Salesforce: 1 year (Preferred);Bachelor's (Preferred);Salesforce: 1 year (Preferred);"

In [144]:
job_detail =''

items = post_soup.find_all('div','jobsearch-JobDescriptionSection-sectionItem')


In [145]:
for i in items:
   
    job_detail += '{};'.format(i.text.strip())
job_detail

'Job TypeContract;'

In [146]:
job_skills = ''
req_quals=post_soup.find_all('li','jobsearch-ReqAndQualSection-item')
for r in req_quals:
    job_skills +='{};'.format(r.text.strip())
job_skills

''

In [147]:
req_quals

[]

In [148]:
job_url

'https://www.indeed.com/viewjob?jk=4c3a3bfe738fd667'

In [149]:
job_description

'Position: ML Engineer\nLocation: California or Remote\nDuration: 9 months (with possibility of extension)\nScheduling Interviews this week\n\nExperience Required: Hiring manager needs someone who has Data Science, ML modeling experience AND engineering / operations experience with model lifecycle management including operationalization of models. In short, ML model development with Python AND MLOps. (This is ML Engineer position instead of pure Data Scientist).\n\nProject Overview:\nThe Machine Learning (ML) Engineer will be a member of Platform Engineering team within Data Science Program helping with ML model development and operationalization.\nRole Scope / Deliverables:\nProvide support to projects including analysis, querying, coding, visualization, modeling, and deploymentBe involved with many aspects of model design and evaluation tasks and prove the efficacy of models built using business driven measurements and sound statistical principlesProvide deep statistical leadership a

In [210]:
job_company = card.find('span','company').text.strip()
job_company

'Specright Inc.'

In [211]:
#card = cards[28]
try: 
    company_url = 'https://www.indeed.com{}'.format(card.find('span','company').a.get('href'))
except AttributeError:
    company_url=''
print(company_url)

https://www.indeed.com/cmp/Twitter


In [277]:
company_url = 'https://www.indeed.com/cmp/Divihn-Integration'#
if len(company_url)>0:
    
    company_response = requests.get(company_url)
    company_soup = BeautifulSoup(company_response.text, 'html.parser')
    try:
        company_industry = company_soup.find('div',text='Industry').find_next_sibling('div').text.strip()
    except AttributeError:
        company_industry = ''
    try:
        company_size =company_soup.find('div',text='Company size').find_next_sibling('div').text.strip()
    except AttributeError:
        company_size = ''
else:
    company_industry = ''
    company_size = ''

print(company_industry, company_size)

Information Technology 51 to 200


In [272]:
company_soup.find('div',text='Industry').find_next_sibling('div').text.strip()

'Retail & Wholesale'

In [269]:
company_soup.find('div',text='Company size').find_next_sibling('div').text.strip()


'51 to 200'

In [196]:
try:
    company_rating = card.find('span','ratingsContent').text.strip()
except AttributeError:
    company_rating = ''
company_rating

'3.9'

In [113]:
job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
job_location

'Remote'

In [114]:
job_summary = card.find('div', 'summary').text.strip()
job_summary

'Scientists will participate in data model development and petabyte-level data process optimization.\nSome experience in big data processing.'

In [115]:
job_post_date = card.find('span','date').text.strip()

In [116]:
today = datetime.today().strftime('%Y-%m-%d') # To do: need to add time 

In [117]:
datetime.today().strftime('%Y_%m_%d_%H_%M_%S')

'2021_02_14_20_13_19'

In [118]:
datetime.today()

datetime.datetime(2021, 2, 14, 20, 13, 19, 452396)

In [119]:
try:
    job_salary = card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary = ''


In [120]:
try:
    job_remote = card.find('span','remote').text.strip()
except AttributeError:
    job_remote = ''

## Generalize the model with a function

In [278]:
from random import randint
def get_record(card):
    """Extract individual job post data from a single record """
    # required variables
    job_id = card.get('data-jk')
    aTag = card.h2.a
    job_title = aTag.get('title')
    
    #job_url = 'https://www.indeed.com' + aTag.get('href')
    job_url = 'https://www.indeed.com/viewjob?jk=' + job_id
       
    job_company = card.find('span','company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    job_summary = card.find('div', 'summary').text.strip()
    job_post_date = card.find('span','date').text.strip()
    
    # optional variables
    try: 
        company_url = 'https://www.indeed.com{}'.format(card.find('span','company').a.get('href'))
    except AttributeError:
        company_url=''
    if len(company_url)>0:    
        company_response = requests.get(company_url)
        company_soup = BeautifulSoup(company_response.text, 'html.parser')
        try:
            company_industry = company_soup.find('div',text='Industry').find_next_sibling('div').text.strip()
        except AttributeError:
            company_industry = ''
        try:
            company_size = company_soup.find('div',text='Company size').find_next_sibling('div').text.strip()
        except AttributeError:
            company_size = ''
        time.sleep(randint(0,3))
    else:
        company_industry = ''
        company_size = ''
        
    try:
        company_rating = card.find('span','ratingsContent').text.strip()
    except AttributeError:
        company_rating = ''
    
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
    
    try:
        job_remote = card.find('span','remote').text.strip()
    except AttributeError:
        job_remote = ''
        
    # To determine if include detailed job decription: only include job description for remote job
    detailInclued = (job_remote =='Remote') | (job_location == 'Remote')
    
    if detailInclued:
        post_response = requests.get(job_url)
        post_soup = BeautifulSoup(post_response.text, 'html.parser')
        
        try:
            job_description = post_soup.find('div','jobsearch-jobDescriptionText').text.strip()
        except AttributeError:
            job_description = ''
                 
        try:
            job_detail = post_soup.find('div','jobsearch-JobDescriptionSection-section').text.strip()
        except:
            job_detail = '' 
         
        time.sleep(randint(1,3))
    else:
        job_description = ''  
        job_detail = ''
        
    today = datetime.today().strftime('%Y-%m-%d')
    
    record = (job_id, job_title,job_company,job_location,company_rating, company_url, company_industry, company_size, job_post_date,today,job_summary,job_salary, job_remote, job_url, job_detail, job_description)
   
   
    return record


In [279]:
len(cards)

54

In [280]:
%time
records = []

for card in cards:
    if records == []:
        record = get_record(card)
        records.append(record)
    else:
        #To check if the card is already stored in the records
        id_array = np.array([r[0] for r in records])
        if not card.get('data-jk') in id_array:   
            record = get_record(card)
            records.append(record)

Wall time: 0 ns


In [281]:
records

[('273e760b11964cb2',
  'Machine Learning Engineer (Remote)',
  'DivIHN Integration, Inc',
  'Deerfield, IL',
  '5.0',
  'https://www.indeed.com/cmp/Divihn-Integration',
  'Information Technology',
  '51 to 200',
  'Just posted',
  '2021-02-27',
  '*This Contract Position is with one of our Direct Clients. Please reach out to Remya - mremya(at)divihn(dot)com for more details about this position. You can…',
  '',
  'Temporarily remote',
  'https://www.indeed.com/viewjob?jk=273e760b11964cb2',
  '',
  ''),
 ('8d52d4557802b691',
  'Data Scientist',
  'CyberCoders',
  'Salt Lake City, UT',
  '3.2',
  'https://www.indeed.com/cmp/Cybercoders',
  'Human Resources & Staffing',
  '201 to 500',
  'Just posted',
  '2021-02-27',
  'Data Scientist -THIS POSITION IS 100% REMOTE- If you are a Data Scientist looking to work on the next big thing in DevOps, please read on! Founded in 2016,…',
  '',
  'Remote',
  'https://www.indeed.com/viewjob?jk=8d52d4557802b691',
  'Job detailsJob TypeFull-time',
  "D

In [401]:
np.array([])

array([], dtype=float64)

In [383]:
id_array, len(records)

(array(['51d2ea04d15fbf1f', '571e360548a8d0c1', '2f143848e64c0573',
        '32e4ffa91d91f8ed', '517da30c1f7ab5f0', 'eb71688408d695be',
        'd2a2d1dba9456a6d', 'd080318aa423d790', '558379216e05a524',
        'd9c74d0d57cd11f2', '9bfea2a8b8774194', 'e874bf89e3161b17',
        'a078eabd7f5b3f38', '1145d029b0d268ae'], dtype='<U10786'),
 15)

In [402]:
type(records[11])

tuple

## Getting the next page


In [233]:
soup

<html>
<head>
<title>hCaptcha solve page</title>
<script async="" defer="" src="https://www.hcaptcha.com/1/api.js"></script>
</head>
<body>
<form action="/jobs?q=machine+learning&amp;fromage=1&amp;start=120" method="POST">
<div class="h-captcha" data-sitekey="eb27f525-f936-43b4-91e2-95a426d4a8bd"></div>
<br/>
<input type="submit" value="Submit"/>
</form>
</body>
</html>

In [406]:
records =[]
url = get_job_search_url('machine learning', 1)
print( url )
while True:
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')   
    cards = soup.find_all('div', 'jobsearch-SerpJobCard')
    print(len(cards))
 
       
    for card in cards:
        if records == []:
            record = get_record(card)
            records.append(record)
        else:
            #To check if the card is already stored in the records
            id_array = np.array([r[0] for r in records])
            if not card.get('data-jk') in id_array:   
                record = get_record(card)
                records.append(record)

    print(len(records))
    if len(cards)> 0:
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
        except AttributeError:
            total_jobs = soup.find('div',{'id':'searchCountPages'}).text.strip()
            print(url)
            print(total_jobs)
            break
        time.sleep(1)
    else: break
        
print(len(records))

https://www.indeed.com/jobs?q=machine learning&fromage=1&limit=50&filter=0
https://www.indeed.com/jobs?q=machine learning&fromage=1&limit=50&filter=0
55
55
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=50
55
110
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=100
55
165
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=150
55
220
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=200
55
275
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=250
55
330
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=300
54
384
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=350
55
439
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=400
55
494
https://www.indeed.com/jobs?q=machine+learning&limit=50&fromage=1&filter=0&start=450
54
548
https://www.indee

In [399]:
np.array([r[0] for r in records])


array(['589dfaeb201f6431', '8e330e086b957051'], dtype='<U16')

In [237]:
soup.find('a', {'aria-label': 'Next'}).get('href')

AttributeError: 'NoneType' object has no attribute 'get'

In [56]:
len(records)

1027

## Putting it all together

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time 
import numpy as np
from random import randint

def get_job_search_url(position, postedDays):
    """Generate a url from position and posted days ago
    with 50 job posts per page and no filter"""

    template = 'https://www.indeed.com/jobs?q={}&fromage={}&limit=50&filter=0'
    url=template.format(position, postedDays)
    
    return url

def get_record(card):
    """Extract individual job post data from a single record """
    # required variables
    job_id = card.get('data-jk')
    aTag = card.h2.a
    job_title = aTag.get('title')
    
    #job_url = 'https://www.indeed.com' + aTag.get('href')
    job_url = 'https://www.indeed.com/viewjob?jk=' + job_id
       
    job_company = card.find('span','company').text.strip()
    job_location = card.find('div', 'recJobLoc').get('data-rc-loc')
    job_summary = card.find('div', 'summary').text.strip()
    job_post_date = card.find('span','date').text.strip()
    
    # optional variables
    try: 
        company_url = 'https://www.indeed.com{}'.format(card.find('span','company').a.get('href'))
    except AttributeError:
        company_url=''
    if len(company_url)>0:    
        company_response = requests.get(company_url)
        company_soup = BeautifulSoup(company_response.text, 'html.parser')
        try:
            company_industry = company_soup.find('div',text='Industry').find_next_sibling('div').text.strip()
        except AttributeError:
            company_industry = ''
        try:
            company_size = company_soup.find('div',text='Company size').find_next_sibling('div').text.strip()
        except AttributeError:
            company_size = ''
        time.sleep(randint(0,1))
    else:
        company_industry = ''
        company_size = ''
        
    try:
        company_rating = card.find('span','ratingsContent').text.strip()
    except AttributeError:
        company_rating = ''
    
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary =''
    
    try:
        job_remote = card.find('span','remote').text.strip()
    except AttributeError:
        job_remote = ''
        
    # To determine if include detailed job decription 
    detailInclued = (job_remote =='Remote') | (job_location == 'Remote')
    
    if detailInclued:
        post_response = requests.get(job_url)
        post_soup = BeautifulSoup(post_response.text, 'html.parser')
        
        try:
            job_description = post_soup.find('div','jobsearch-jobDescriptionText').text.strip()
        except AttributeError:
            job_description = ''
        try:
            job_detail = post_soup.find('div','jobsearch-JobDescriptionSection').text.strip()    
        except AttributeError:
            job_detail = ''
               
        time.sleep(randint(1,3))
    else:
        job_description = ''   
        job_detail = ''
        
    today = datetime.today().strftime('%Y-%m-%d')
    
    record = (job_id, job_title,job_company,job_location,company_rating, company_url, company_industry, company_size, job_post_date,today,job_summary,job_salary, job_remote, job_url, job_detail, job_description)
   
    return record

def main(position, postedDay, fileName):
    """Run the main program routine"""
    records = []
    url = get_job_search_url(position, postedDay)

    # extract the job data
    while True:        
        print(url)
       
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        if soup.title.text.find('Captcha') != -1: 
            print('blocked by Captcha!')
            break
            
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')        
        total_jobs = soup.find('div',{'id':'searchCountPages'}).text.strip()
        print(len(cards), total_jobs)
        
           
        for card in cards:
            if records == []:
                record = get_record(card)
                records.append(record)
            else:
                #To check if the card is already stored in the records to reduce the duplicate records
                id_array = np.array([r[0] for r in records])
                if not card.get('data-jk') in id_array:   
                    record = get_record(card)
                    records.append(record)
   
        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href') 
        except AttributeError:     
            print('done!')
            break
   
    firstRow ='{0}: {1} {2}'.format(position, str(len(records)),total_jobs[total_jobs.index('of'):])
    print(firstRow)
    # save the job data
    if (len(records) > 0):
        fn = '../data/raw/{0}{1}.csv'.format(fileName,datetime.today().strftime('%Y_%m_%d'))
            
        with open(fn, 'w', newline ='', encoding ='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(firstRow)
            writer.writerow(['JobID', 'JobTitle', 'Company','Location', 'CompanyRating', 'CompanyUrl', 'CompanyIndustry','CompanySize', 'PostDate', 'ExtractDate','Summary', 'Salary', 'Remote','JobUrl','JobDetail', 'JobDescription'])
            writer.writerows(records)

Notes:

1: Indeed will not return exactly amount of jobs shown on the pagination based on some internal filtering they run, unless you add query string &filter = 0 in the url.  With the filter, it will still only can get to 20*50 =1000 records 
https://www.indeed.com/jobs?q=machine+learning&fromage=1&filter=0&start=200
"We have removed 2,463 job postings very similar to those already shown. To see these additional results, you may repeat your search with the omitted job postings included." 

2: Every midnight, run program for 3 search criterion without job detailed description for 1 day posting without filter
Observesd: the maximum number of records Indeed allow to get is 50 * 20 = 1000

3: With job detailed description, need to added some sleep time to avoid being blocked by CAPTCHA
Also, need to come up some algorithm to random sample the detailed job description, considering only scrape it of remote jobs


4: sometimes may encounter connection broken from Request

5: Modified the search string by removing the ',' in order to narrow the result according to the Law of Diminishing Marginal Utility.    2/19/21



In [2]:
# To do list:
# 1: Add a scheduler 
# 2: Add algorithm for random sampling for detailed job description : 
# option1: include detailed description only for remote job:             done!
# option2: include detailed description only for several companies
# 3: Add a log writer
# 4: Add company industry info

# LOG:
#2/23/2021(T):'data science','data analytics data analyst','machine learning','artificial intelligence'. 
# 4 files with description on remote jobs.  Run at 8AM 
#2/24/2021(W):'"data science"','"data analyst"','"machine learning"','"artificial intelligence"'.
# 4 files with description on remote jobs.  Run at 8AM 
#2/25/2021(TH):same as before, run at 8Am, 2863 records
#2/27/2021(Sat.): same as before, run at 9am. Add companyUrl, companyIndustry, companySize



In [3]:
# run the main program

#time.sleep(10000)
main('"data science"',1 ,'ds_last1d_RC') 

main('"data analyst"',1 ,'da_last1d_RC_')     

main('"machine learning"', 1 ,'ml_last1d_RC_')   

main('"artificial intelligence"', 1, 'ai_last1d_RC_') 

 


# add discription 


https://www.indeed.com/jobs?q="data science"&fromage=1&limit=50&filter=0
53 Page 1 of 989 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=50
56 Page 2 of 989 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=100
55 Page 3 of 990 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=150
55 Page 4 of 990 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=200
55 Page 5 of 990 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=250
56 Page 6 of 990 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=300
54 Page 7 of 990 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=350
55 Page 8 of 990 jobs
https://www.indeed.com/jobs?q=%22data+science%22&limit=50&fromage=1&filter=0&start=400
55 Page 9 of 990 jobs
https://www.indeed.com/jobs?q=%22d